In [1]:
import math

import cvxpy as cp
import scipy as sp
import numpy as np
import pickle

In [2]:
cp.installed_solvers()

['CLARABEL', 'ECOS', 'ECOS_BB', 'OSQP', 'SCIPY', 'SCS']

In [3]:
N = 100
M = 100
max_assigned = math.ceil(N / M)
np.random.seed(0)
W = sp.stats.lognorm.rvs(s=1, size=(N, M))
W

array([[ 5.83603919,  1.49205924,  2.66109578, ...,  5.96476998,
         1.13531721,  1.49479543],
       [ 6.57418553,  0.25982185,  0.28069545, ...,  2.27846997,
         8.69924247,  3.80580659],
       [ 0.69129969,  0.78711637,  3.00314357, ...,  1.79132161,
         0.67068947,  1.44781553],
       ...,
       [ 0.94625443,  1.13956091,  1.55378061, ...,  2.64377323,
         0.39893752,  1.94107219],
       [ 0.87503499,  0.20880169,  0.17400845, ...,  1.07341762,
        11.42413823,  2.6423831 ],
       [ 0.39338669, 17.55262551,  0.16661859, ...,  1.67677479,
         0.9676153 ,  3.66237349]], shape=(100, 100))

In [4]:
x = cp.Variable((N, M), 'x')
w = cp.Parameter((N, M), name='w')
constraints = [
    cp.sum(x, 0) <= max_assigned,   # enforce even distribution
    cp.sum(x, 1) == 1,
    0 <= x,
]
problem = cp.Problem(cp.Minimize(cp.vdot(w, x)), constraints)

# test and compile
def solve_cvxpy():
    w.value = sp.stats.lognorm.rvs(s=1, size=(N, M))
    problem.solve(solver='OSQP')
    return x.value

solve_cvxpy().argmax(1)

C:\Users\volke\AppData\Local\pypoetry\Cache\virtualenvs\phantom-sc2-ZmbTkktB-py3.12\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:254: UserWarning: Your problem has too many parameters for efficient DPP compilation. We suggest setting 'ignore_dpp = True'.
  warnings.warn(


array([96, 84, 12,  3, 63, 11, 76,  9, 50, 36, 59,  1, 81, 52,  6, 42, 31,
       79, 97, 68, 86, 58, 54, 83, 91,  4, 25, 43, 71, 47, 33, 64, 99, 62,
       78, 16, 44, 60, 15,  2, 70, 57, 21, 98, 82, 94, 10, 67, 66, 69, 19,
       95, 55, 38, 30, 61, 37, 85, 14, 88,  0, 53, 41, 13, 28, 48, 93, 87,
       46, 75, 72, 65,  7, 77, 90, 45, 23, 51, 34,  5, 39, 35, 73, 92, 22,
       56, 29, 18, 20, 74, 40,  8, 89, 24, 17, 32, 26, 80, 49, 27])

In [5]:
A_ub = np.tile(np.identity(M), (1, N))
b_ub = np.full(M, max_assigned)

A_eq = np.repeat(np.identity(N), M, axis=1)
b_eq = np.full(N, 1.0)

c = W.flatten()

In [6]:
def solve_highs():
    return sp.optimize.linprog(
        c=c,
        A_ub=A_ub,
        b_ub=b_ub,
        A_eq=A_eq,
        b_eq=b_eq,
        method="highs",
    ).x.reshape((N, M))
solve_highs().argmax(1)

array([33, 40, 78, 34, 22, 89, 51, 35, 50, 13, 39, 29, 56, 97, 46, 83, 63,
       86, 90, 11, 18, 14, 54, 60, 72,  7, 92,  0,  6, 48, 20, 24,  3, 70,
       62, 45, 79, 52, 53, 12,  9, 21, 38, 37, 69, 82, 74, 93, 49, 10, 23,
       91, 31, 28, 57, 94, 85, 76,  4, 26, 15, 64, 47, 55, 44, 30,  5, 27,
       84, 61, 65,  1, 19, 25, 32, 77, 95, 98, 99, 80, 42, 75, 16, 73, 59,
       17, 41, 81, 87, 71, 96, 88, 43, 36, 58, 68, 66, 67,  8,  2])

In [7]:
A_ub_sparse = sp.sparse.csr_matrix(A_ub)
A_eq_sparse = sp.sparse.csr_matrix(A_eq)

In [8]:
def solve_interior_point():
    return sp.optimize.linprog(
        c=c,
        A_ub=A_ub,
        b_ub=b_ub,
        A_eq=A_eq,
        b_eq=b_eq,
        method="interior-point",
        options=dict(sparse=True),
    ).x.reshape((N, M))
solve_interior_point().argmax(1)

C:\Users\volke\AppData\Local\Temp\ipykernel_33256\3670224332.py:2: DeprecationWarning: `method='interior-point'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  return sp.optimize.linprog(


array([33, 40, 78, 34, 22, 89, 51, 35, 50, 13, 39, 29, 56, 97, 46, 83, 63,
       86, 90, 11, 18, 14, 54, 60, 72,  7, 92,  0,  6, 48, 20, 24,  3, 70,
       62, 45, 79, 52, 53, 12,  9, 21, 38, 37, 69, 82, 74, 93, 49, 10, 23,
       91, 31, 28, 57, 94, 85, 76,  4, 26, 15, 64, 47, 55, 44, 30,  5, 27,
       84, 61, 65,  1, 19, 25, 32, 77, 95, 98, 99, 80, 42, 75, 16, 73, 59,
       17, 41, 81, 87, 71, 96, 88, 43, 36, 58, 68, 66, 67,  8,  2])

In [9]:
%%timeit
solve_cvxpy()

The slowest run took 5.07 times longer than the fastest. This could mean that an intermediate result is being cached.
484 ms ± 308 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
solve_highs()

30.6 ms ± 820 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit
solve_interior_point()

C:\Users\volke\AppData\Local\Temp\ipykernel_33256\3670224332.py:2: DeprecationWarning: `method='interior-point'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  return sp.optimize.linprog(


161 ms ± 9.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
